In [ ]:
from mo_figs import *
# from fourierDistill import *
# from subset_predictors import *
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.linear_model import MultiTaskElasticNetCV, RidgeCV, MultiTaskElasticNet, LogisticRegressionCV

from subset_predictors import L0L2RegressorCV
from fourierDistill import *

In [ ]:
def load_csvs(path):

    X_train = pd.read_csv(f'{path}/X_trainval.csv', index_col=0)
    X_train_hat = pd.read_csv(f'{path}/X_trainval_hat.csv', index_col=0)
    X_test = pd.read_csv(f'{path}/X_test.csv', index_col=0)
    X_test_hat = pd.read_csv(f'{path}/X_test_hat.csv', index_col=0)
    y_train = pd.read_csv(f'{path}/y_trainval.csv', index_col=0)
    y_train_hat = pd.read_csv(f'{path}/y_trainval_hat.csv', index_col=0)
    y_test = pd.read_csv(f'{path}/y_test.csv', index_col=0)
    y_test_hat = pd.read_csv(f'{path}/y_test_hat.csv', index_col=0)

    return X_train, X_train_hat, X_test, X_test_hat, y_train, y_train_hat, y_test, y_test_hat

In [ ]:
X_train, X_train_hat, X_test, X_test_hat, y_train, y_train_hat, y_test, y_test_hat = load_csvs(f'/home/mattyshen/interpretableDistillation/interpretDistill/data/cbm_datasets/seed0_Joint0.01SigmoidModel__Seed1')

In [ ]:
X_train_hat

In [ ]:
ftd = FTDistillRegressorCV(pre_interaction='None', size_interactions=3, post_interaction='l0l2', post_max_features=3, mo=True, cv=1)

In [ ]:
ftd.pre_interaction_model

In [ ]:
sub = L0L2RegressorCV()

In [ ]:
sub.fit(X_train_hat, y_train)

In [ ]:
ftd.fit((X_train_hat > 0.5).astype(int), y_train_hat)

In [ ]:
logreg = LogisticRegressionCV(Cs = np.arange(1, 6),l1_ratios=[0.5], cv=3, penalty='elasticnet', solver='saga', max_iter=100)

In [ ]:
logreg.fit(X_train_hat, y_train_hat.idxmax(axis=1).astype(int))

In [ ]:
accuracy_score(logreg.predict(X_train_hat), y_train_hat.idxmax(axis=1).astype(int))

In [ ]:
from sklearn.multioutput import MultiOutputRegressor

In [ ]:
multi_l0l2 = MultiOutputRegressor(
    L0L2RegressorCV(max_support_size = 5, cv=3)
)

In [ ]:
multi_l0l2.fit(X_train_hat, y_train_hat)

In [ ]:
accuracy_score(np.argmax(multi_l0l2.predict(X_test_hat), axis = 1), y_test_hat.idxmax(axis=1).astype(int))

In [ ]:
EN = MultiTaskElasticNetCV(cv = 3, max_iter=100)

In [ ]:
EN_fixed = MultiTaskElasticNet(alpha = 3, max_iter=100)

In [ ]:
Ridge = RidgeCV()

In [ ]:
EN.fit(X_train_hat, y_train_hat)

In [ ]:
EN_fixed.fit(X_train_hat, y_train_hat)

In [ ]:
Ridge.fit(X_train_hat, y_train_hat)

In [ ]:
np.sum(EN_fixed.coef_ == 0, axis = 1)

In [ ]:
np.where(EN_fixed.coef_ ==0)

In [ ]:
plt.imshow(EN_fixed.coef_)

In [ ]:
accuracy_score(EN.predict(X_test_hat).argmax(axis=1).astype(int), y_test_hat.idxmax(axis=1).astype(int))

In [ ]:
accuracy_score(EN_fixed.predict(X_test_hat).argmax(axis=1).astype(int), y_test_hat.idxmax(axis=1).astype(int))

In [ ]:
accuracy_score(Ridge.predict(X_test_hat).argmax(axis=1).astype(int), y_test_hat.idxmax(axis=1).astype(int))

In [ ]:
mean_squared_error(y_train_hat.to_numpy(), Ridge.predict(X_train_hat)), mean_squared_error(y_test_hat.to_numpy(), Ridge.predict(X_test_hat))

In [ ]:
accuracy_score(y_test_hat.idxmax(axis=1).astype(int), y_test)

In [ ]:
y_train_hat.idxmax(axis=1).astype(int)